In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import cv2
from glasses.nn.models import *
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from glasses.utils.Storage import ForwardModuleStorage, BackwardModuleStorage
from PIL import Image
from io import BytesIO
import requests
from torchvision.transforms import Normalize

r = requests.get('https://i.insider.com/5df126b679d7570ad2044f3e?width=700&format=jpeg&auto=webp')
im = Image.open(BytesIO(r.content))

In [4]:
EfficientNet.efficientnet_b4(pretrained=True, version='tf', padding='same')

INFO:root:Loaded efficientnet_b4_tf pretrained weights.


EfficientNet(
  (encoder): EfficientNetEncoder(
    (gate): ConvBnAct(
      (conv): Conv2dPad(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Swish()
    )
    (blocks): ModuleList(
      (0): EfficientNetLayer(
        (block): Sequential(
          (0): EfficientNetBasicBlock(
            (block): Residual(
              (block): Sequential(
                (depth): ConvBnAct(
                  (conv): DepthWiseConv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=('same', 'same'), groups=48, bias=False)
                  (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (act): Swish()
                )
                (point): Sequential(
                  (0): ChannelSE(
                    (avg_pool): AdaptiveAvgPool2d(output_size=1)
                    (att): Sequential(
                      (con